# Data Processing Notebook
This notebook loads the pre-parsed NDJSON (one trial per line) and applies basic cleaning
steps so downstream modeling has consistent types.

## Step 1 · Load the NDJSON dataset
Use pandas to read the pre-parsed trials data so each row corresponds to a single study.


In [ ]:
# Load libraries needed for data IO and wrangling
import json
from pathlib import Path
import pandas as pd

# Read the NDJSON export (one trial per line) into a DataFrame
DATA_PATH = Path('../data/trials_summary.ndjson')
assert DATA_PATH.exists(), f'{DATA_PATH} not found'
df = pd.read_json(DATA_PATH, lines=True)

# Quick peek to confirm the schema loaded as expected
df.head()

,xml_path,nct_id,org_study_id,brief_title,official_title,overall_status,why_stopped,phase,study_type,lead_sponsor,...,conditions,condition_mesh_terms,keywords,interventions,intervention_mesh_terms,primary_outcomes,secondary_outcomes,number_of_arms,number_of_groups,locations
0,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000102,NCRR-M01RR01070-0506,Congenital Adrenal Hyperplasia: Calcium Channe...,,Completed,,Phase 1/Phase 2,Interventional,National Center for Research Resources (NCRR),...,[Congenital Adrenal Hyperplasia],"[Adrenal Hyperplasia, Congenital]",[],"[{'type': 'Drug', 'name': 'Nifedipine', 'descr...",[Nifedipine],[],[],,,"{'facility_count': 1, 'countries': ['United St..."
1,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000104,NCRR-M01RR00400-0587,Does Lead Burden Alter Neuropsychological Deve...,,Completed,,,Observational,National Center for Research Resources (NCRR),...,[Lead Poisoning],[Lead Poisoning],[lead overburden],"[{'type': 'Procedure', 'name': 'ERP measures o...",[],[],[],,,"{'facility_count': 1, 'countries': ['United St..."
2,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000105,2002LS032,Vaccination With Tetanus and KLH to Assess Imm...,Vaccination With Tetanus Toxoid and Keyhole Li...,Terminated,Replaced by another study.,,Observational,"Masonic Cancer Center, University of Minnesota",...,[Cancer],[Neoplasms],[],"[{'type': 'Biological', 'name': 'Intracel KLH ...","[keyhole-limpet hemocyanin, montanide ISA 51, ...",[To assess whether patients can mediate an app...,[Tetanus Response],,3,"{'facility_count': 1, 'countries': ['United St..."
3,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000106,NCRR-M01RR03186-9943,41.8 Degree Centigrade Whole Body Hyperthermia...,,Unknown status,,N/A,Interventional,National Center for Research Resources (NCRR),...,[Rheumatic Diseases],[Rheumatic Diseases],[Rheumatoid Diseases],"[{'type': 'Device', 'name': 'Whole body hypert...",[],[],[],,,"{'facility_count': 1, 'countries': ['United St..."
4,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000107,NCRR-M01RR00109-0737,Body Water Content in Cyanotic Congenital Hear...,,Completed,,,Observational,National Center for Research Resources (NCRR),...,"[Heart Defects, Congenital]","[Heart Defects, Congenital]",[Cyanotic Congenital Heart Disease],[],[],[],[],,,"{'facility_count': 1, 'countries': ['United St..."


## Step 2 · Normalize date columns
Convert all relevant date strings into pandas `datetime` objects for consistent comparisons later.


In [ ]:
# Convert the most useful date columns into pandas datetime objects
DATE_COLS = [
    "study_first_posted",
    "last_update_posted",
    "start_date",
    "completion_date",
    "primary_completion_date",
]
for col in DATE_COLS:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df[DATE_COLS].head()


/var/folders/g_/964t0wy17cnbgv5b32sc9sv80000gn/T/ipykernel_47307/2790210628.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
/var/folders/g_/964t0wy17cnbgv5b32sc9sv80000gn/T/ipykernel_47307/2790210628.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
/var/folders/g_/964t0wy17cnbgv5b32sc9sv80000gn/T/ipykernel_47307/2790210628.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


,xml_path,nct_id,org_study_id,brief_title,official_title,overall_status,why_stopped,phase,study_type,lead_sponsor,...,primary_outcomes,secondary_outcomes,number_of_arms,number_of_groups,locations,minimum_age_years,maximum_age_years,enrollment_num,facility_count,conditions_count
0,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000102,NCRR-M01RR01070-0506,Congenital Adrenal Hyperplasia: Calcium Channe...,,Completed,,Phase 1/Phase 2,Interventional,National Center for Research Resources (NCRR),...,[],[],,,"{'facility_count': 1, 'countries': ['United St...",14.0,35.0,NaN,1,1
1,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000104,NCRR-M01RR00400-0587,Does Lead Burden Alter Neuropsychological Deve...,,Completed,,,Observational,National Center for Research Resources (NCRR),...,[],[],,,"{'facility_count': 1, 'countries': ['United St...",0.0,NaN,NaN,1,1
2,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000105,2002LS032,Vaccination With Tetanus and KLH to Assess Imm...,Vaccination With Tetanus Toxoid and Keyhole Li...,Terminated,Replaced by another study.,,Observational,"Masonic Cancer Center, University of Minnesota",...,[To assess whether patients can mediate an app...,[Tetanus Response],,3,"{'facility_count': 1, 'countries': ['United St...",18.0,NaN,112.0,1,1
3,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000106,NCRR-M01RR03186-9943,41.8 Degree Centigrade Whole Body Hyperthermia...,,Unknown status,,N/A,Interventional,National Center for Research Resources (NCRR),...,[],[],,,"{'facility_count': 1, 'countries': ['United St...",18.0,65.0,NaN,1,1
4,/Users/leo/Desktop/520 Project 2/ctg-public-xm...,NCT00000107,NCRR-M01RR00109-0737,Body Water Content in Cyanotic Congenital Hear...,,Completed,,,Observational,National Center for Research Resources (NCRR),...,[],[],,,"{'facility_count': 1, 'countries': ['United St...",17.0,60.0,NaN,1,1


## Step 3 · Engineer numeric-friendly columns
Extract numerical representations for ages, enrollment counts, facility coverage, and condition breadth.


In [ ]:
import re
import numpy as np

# Convert textual age expressions ("18 Years", "6 Months") into numeric years
AGE_PATTERN = re.compile(r"(\d+)")


def _age_to_years(value):
    if pd.isna(value) or value in ("", "N/A"):
        return np.nan
    text = str(value)
    match = AGE_PATTERN.search(text)
    if not match:
        return np.nan
    number = float(match.group(1))
    text_lower = text.lower()
    if "month" in text_lower:
        return number / 12
    if "week" in text_lower:
        return number / 52
    if "day" in text_lower:
        return number / 365
    return number


df["minimum_age_years"] = df["minimum_age"].apply(_age_to_years)
df["maximum_age_years"] = df["maximum_age"].apply(_age_to_years)

# Enrollment counts sometimes come as strings; coerce to numeric
df["enrollment_num"] = pd.to_numeric(df["enrollment"], errors="coerce")

# Pull facility counts out of the nested `locations` dict

def _extract_facility_count(value):
    if isinstance(value, dict):
        return value.get("facility_count")
    return np.nan


df["facility_count"] = df["locations"].apply(_extract_facility_count)

# Track how many conditions are listed per trial

df["conditions_count"] = df["conditions"].apply(lambda x: len(x) if isinstance(x, list) else 0)

df[[
    "minimum_age_years",
    "maximum_age_years",
    "enrollment_num",
    "facility_count",
    "conditions_count",
]].head()


## Step 4 · Quick sanity checks
Review descriptive statistics for the engineered numeric columns to spot obvious anomalies.


In [ ]:
numeric_cols = [
    "minimum_age_years",
    "maximum_age_years",
    "enrollment_num",
    "facility_count",
    "conditions_count",
]

# Display descriptive stats (count, mean, percentiles, etc.)
df[numeric_cols].describe()


## Step 5 · Persist the cleaned table
Write the processed DataFrame to Parquet so downstream notebooks can consume a compact, typed dataset.


In [ ]:
OUTPUT_PATH = Path("../data/trials_summary_clean.parquet")

# Persist cleaned features for future modeling notebooks
df.to_parquet(OUTPUT_PATH, index=False)
OUTPUT_PATH
